***GENERATED CODE FOR life PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Product_Info_2', 'transformation_label': 'String Indexer'}], 'feature': 'Product_Info_2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'A1', 'max': 'E1', 'missing': '0', 'distinct': '18'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Product_Info_2'}, {'feature_label': 'Product_Info_2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Product_Info_2')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run lifeHooks.ipynb
try:
	#sourcePreExecutionHook()

	lifeiurancedatatet = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/testdata/1712537935067_lifeisurancedatatest.csv', 'filename': 'lifeisurancedatatest.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/test/lifeisurancedatatest.csv', 'viewFileName': 'lifeisurancedatatest.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://dnm.bfirst.ai:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run lifeHooks.ipynb
try:
	#transformationPreExecutionHook()

	lifeautofe = TransformationMain.run(lifeiurancedatatet,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "Id", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "15514.27", "stddev": "196.65", "min": "15180", "max": "15857", "missing": "0"}, "updatedLabel": "Id"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Product_Info_1", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.03", "stddev": "0.16", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "Product_Info_1"}, {"transformationsData": [{"feature_label": "Product_Info_2", "transformation_label": "String Indexer"}], "feature": "Product_Info_2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "A1", "max": "E1", "missing": "0", "distinct": "18"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Product_Info_2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Product_Info_3", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "24.4", "stddev": "5.32", "min": "4", "max": "37", "missing": "0"}, "updatedLabel": "Product_Info_3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Product_Info_4", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.32", "stddev": "0.28", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Product_Info_4"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Product_Info_5", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.0", "stddev": "0.04", "min": "2", "max": "3", "missing": "0"}, "updatedLabel": "Product_Info_5"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Product_Info_6", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.7", "stddev": "0.71", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Product_Info_6"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Product_Info_7", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.02", "stddev": "0.18", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Product_Info_7"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Ins_Age", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.4", "stddev": "0.2", "min": "0.029850746", "max": "0.850746269", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Ins_Age"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Ht", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.7", "stddev": "0.07", "min": "0.454545455", "max": "0.890909091", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Ht"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Wt", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.29", "stddev": "0.09", "min": "0.121338912", "max": "0.623430962", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Wt"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BMI", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.47", "stddev": "0.12", "min": "0.182548053", "max": "0.99539889", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "BMI"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Employment_Info_1", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "0.08", "stddev": "0.08", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Employment_Info_1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Employment_Info_2", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "8.7", "stddev": "4.25", "min": "1", "max": "36", "missing": "0"}, "updatedLabel": "Employment_Info_2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Employment_Info_3", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.36", "stddev": "0.77", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Employment_Info_3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Employment_Info_4", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "448", "mean": "0.01", "stddev": "0.02", "min": "0.0", "max": "0.3", "missing": "62"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Employment_Info_4"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Employment_Info_5", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.14", "stddev": "0.35", "min": "2", "max": "3", "missing": "0"}, "updatedLabel": "Employment_Info_5"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Employment_Info_6", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "406", "mean": "0.37", "stddev": "0.36", "min": "0.0", "max": "1.0", "missing": "90"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Employment_Info_6"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InsuredInfo_1", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.21", "stddev": "0.4", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "InsuredInfo_1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InsuredInfo_2", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.01", "stddev": "0.08", "min": "2", "max": "3", "missing": "0"}, "updatedLabel": "InsuredInfo_2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InsuredInfo_3", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "5.85", "stddev": "2.59", "min": "1", "max": "11", "missing": "0"}, "updatedLabel": "InsuredInfo_3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InsuredInfo_4", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.88", "stddev": "0.33", "min": "2", "max": "3", "missing": "0"}, "updatedLabel": "InsuredInfo_4"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InsuredInfo_5", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.02", "stddev": "0.2", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "InsuredInfo_5"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InsuredInfo_6", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.41", "stddev": "0.49", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "InsuredInfo_6"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InsuredInfo_7", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.04", "stddev": "0.29", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "InsuredInfo_7"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Insurance_History_1", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.74", "stddev": "0.44", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "Insurance_History_1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Insurance_History_2", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.06", "stddev": "0.33", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Insurance_History_2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Insurance_History_3", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.18", "stddev": "0.99", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Insurance_History_3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Insurance_History_4", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.94", "stddev": "0.94", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Insurance_History_4"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Insurance_History_5", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "273", "mean": "0.0", "stddev": "0.0", "min": "2.3E-5", "max": "0.016666667", "missing": "227"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Insurance_History_5"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Insurance_History_7", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.9", "stddev": "0.96", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Insurance_History_7"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Insurance_History_8", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.07", "stddev": "0.76", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Insurance_History_8"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Insurance_History_9", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.41", "stddev": "0.51", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Insurance_History_9"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Family_Hist_1", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.69", "stddev": "0.47", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Family_Hist_1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Family_Hist_2", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "258", "mean": "0.48", "stddev": "0.15", "min": "0.130434783", "max": "0.898550725", "missing": "242"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Family_Hist_2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Family_Hist_3", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "228", "mean": "0.49", "stddev": "0.14", "min": "0.058823529", "max": "1.0", "missing": "284"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Family_Hist_3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Family_Hist_4", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "335", "mean": "0.44", "stddev": "0.16", "min": "0.084507042", "max": "0.845070423", "missing": "152"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Family_Hist_4"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Family_Hist_5", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "138", "mean": "0.49", "stddev": "0.13", "min": "0.1875", "max": "0.758928571", "missing": "362"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Family_Hist_5"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_1", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "431", "mean": "7.84", "stddev": "14.14", "min": "0", "max": "171", "missing": "68"}, "updatedLabel": "Medical_History_1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_2", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "243.62", "stddev": "171.77", "min": "3", "max": "631", "missing": "0"}, "updatedLabel": "Medical_History_2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_3", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.12", "stddev": "0.33", "min": "2", "max": "3", "missing": "0"}, "updatedLabel": "Medical_History_3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_4", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.69", "stddev": "0.46", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "Medical_History_4"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_5", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.01", "stddev": "0.1", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "Medical_History_5"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_6", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.88", "stddev": "0.48", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Medical_History_6"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_7", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.01", "stddev": "0.16", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Medical_History_7"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_8", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.04", "stddev": "0.28", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Medical_History_8"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_9", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.77", "stddev": "0.42", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "Medical_History_9"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_10", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "8", "mean": "180.88", "stddev": "109.49", "min": "1", "max": "240", "missing": "492"}, "updatedLabel": "Medical_History_10"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_11", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "3.0", "stddev": "0.09", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Medical_History_11"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_12", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.06", "stddev": "0.24", "min": "2", "max": "3", "missing": "0"}, "updatedLabel": "Medical_History_12"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_13", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.73", "stddev": "0.69", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Medical_History_13"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_14", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.97", "stddev": "0.21", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Medical_History_14"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_15", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "121", "mean": "127.72", "stddev": "100.59", "min": "0", "max": "240", "missing": "379"}, "updatedLabel": "Medical_History_15"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_16", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.31", "stddev": "0.73", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Medical_History_16"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_17", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.98", "stddev": "0.13", "min": "2", "max": "3", "missing": "0"}, "updatedLabel": "Medical_History_17"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_18", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.07", "stddev": "0.25", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "Medical_History_18"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_19", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.03", "stddev": "0.18", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "Medical_History_19"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_20", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.99", "stddev": "0.08", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "Medical_History_20"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_21", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.09", "stddev": "0.29", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "Medical_History_21"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_22", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.98", "stddev": "0.15", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "Medical_History_22"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_23", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.53", "stddev": "0.85", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Medical_History_23"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_24", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "38", "mean": "34.97", "stddev": "55.39", "min": "0", "max": "240", "missing": "473"}, "updatedLabel": "Medical_History_24"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_25", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.18", "stddev": "0.4", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Medical_History_25"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_26", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.82", "stddev": "0.38", "min": "2", "max": "3", "missing": "0"}, "updatedLabel": "Medical_History_26"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_27", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.98", "stddev": "0.22", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Medical_History_27"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_28", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.08", "stddev": "0.27", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "Medical_History_28"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_29", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.54", "stddev": "0.84", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Medical_History_29"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_30", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.03", "stddev": "0.18", "min": "2", "max": "3", "missing": "0"}, "updatedLabel": "Medical_History_30"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_31", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.98", "stddev": "0.2", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Medical_History_31"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_32", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "10", "mean": "4.5", "stddev": "11.53", "min": "0", "max": "37", "missing": "485"}, "updatedLabel": "Medical_History_32"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_33", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.84", "stddev": "0.54", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Medical_History_33"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_34", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.7", "stddev": "0.71", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Medical_History_34"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_35", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.0", "stddev": "0.0", "min": "1", "max": "1", "missing": "0"}, "updatedLabel": "Medical_History_35"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_36", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.22", "stddev": "0.44", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Medical_History_36"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_37", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.94", "stddev": "0.23", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "Medical_History_37"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_38", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.0", "stddev": "0.06", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "Medical_History_38"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_39", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.84", "stddev": "0.54", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Medical_History_39"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_40", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.96", "stddev": "0.29", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Medical_History_40"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_History_41", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.64", "stddev": "0.93", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Medical_History_41"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_1", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.03", "stddev": "0.18", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_2", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.1", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_3", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.06", "stddev": "0.23", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_4", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.1", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_4"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_5", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.1", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_5"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_6", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.09", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_6"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_7", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.08", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_7"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_8", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.09", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_8"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_9", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.08", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_9"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_10", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.03", "stddev": "0.18", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_10"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_11", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.05", "stddev": "0.22", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_11"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_12", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.08", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_12"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_13", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.06", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_13"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_14", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.08", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_14"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_15", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.19", "stddev": "0.39", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_15"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_16", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.11", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_16"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_17", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.1", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_17"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_18", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.08", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_18"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_19", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.09", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_19"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_20", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.09", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_20"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_21", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.11", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_21"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_22", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.03", "stddev": "0.16", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_22"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_23", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.1", "stddev": "0.29", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_23"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_24", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.02", "stddev": "0.13", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_24"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_25", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.09", "stddev": "0.29", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_25"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_26", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.11", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_26"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_27", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.1", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_27"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_28", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.02", "stddev": "0.14", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_28"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_29", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.06", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_29"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_30", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.02", "stddev": "0.15", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_30"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_31", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.09", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_31"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_32", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.02", "stddev": "0.15", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_32"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_33", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.02", "stddev": "0.14", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_33"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_34", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.02", "stddev": "0.14", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_34"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_35", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.11", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_35"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_36", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.08", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_36"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_37", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.07", "stddev": "0.26", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_37"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_38", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.09", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_38"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_39", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.02", "stddev": "0.13", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_39"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_40", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.05", "stddev": "0.22", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_40"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_41", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.11", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_41"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_42", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.04", "stddev": "0.21", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_42"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_43", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.11", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_43"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_44", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.1", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_44"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_45", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.11", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_45"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_46", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.04", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_46"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_47", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.02", "stddev": "0.14", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_47"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Medical_Keyword_48", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.06", "stddev": "0.23", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Medical_Keyword_48"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Response", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "5.58", "stddev": "2.55", "min": "1", "max": "8", "missing": "0"}, "updatedLabel": "Response"}]}))

	#transformationPostExecutionHook(lifeautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run lifeHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(lifeautofe, ["Id", "Product_Info_1", "Product_Info_3", "Product_Info_4", "Product_Info_5", "Product_Info_6", "Product_Info_7", "Ins_Age", "Ht", "Wt", "BMI", "Employment_Info_1", "Employment_Info_2", "Employment_Info_3", "Employment_Info_4", "Employment_Info_5", "Employment_Info_6", "InsuredInfo_1", "InsuredInfo_2", "InsuredInfo_3", "InsuredInfo_4", "InsuredInfo_5", "InsuredInfo_6", "InsuredInfo_7", "Insurance_History_1", "Insurance_History_2", "Insurance_History_3", "Insurance_History_4", "Insurance_History_5", "Insurance_History_7", "Insurance_History_8", "Insurance_History_9", "Family_Hist_1", "Family_Hist_2", "Family_Hist_3", "Family_Hist_4", "Family_Hist_5", "Medical_History_1", "Medical_History_2", "Medical_History_3", "Medical_History_4", "Medical_History_5", "Medical_History_6", "Medical_History_7", "Medical_History_8", "Medical_History_9", "Medical_History_10", "Medical_History_11", "Medical_History_12", "Medical_History_13", "Medical_History_14", "Medical_History_15", "Medical_History_16", "Medical_History_17", "Medical_History_18", "Medical_History_19", "Medical_History_20", "Medical_History_21", "Medical_History_22", "Medical_History_23", "Medical_History_24", "Medical_History_25", "Medical_History_26", "Medical_History_27", "Medical_History_28", "Medical_History_29", "Medical_History_30", "Medical_History_31", "Medical_History_32", "Medical_History_33", "Medical_History_34", "Medical_History_35", "Medical_History_36", "Medical_History_37", "Medical_History_38", "Medical_History_39", "Medical_History_40", "Medical_History_41", "Medical_Keyword_1", "Medical_Keyword_2", "Medical_Keyword_3", "Medical_Keyword_4", "Medical_Keyword_5", "Medical_Keyword_6", "Medical_Keyword_7", "Medical_Keyword_8", "Medical_Keyword_9", "Medical_Keyword_10", "Medical_Keyword_11", "Medical_Keyword_12", "Medical_Keyword_13", "Medical_Keyword_14", "Medical_Keyword_15", "Medical_Keyword_16", "Medical_Keyword_17", "Medical_Keyword_18", "Medical_Keyword_19", "Medical_Keyword_20", "Medical_Keyword_21", "Medical_Keyword_22", "Medical_Keyword_23", "Medical_Keyword_24", "Medical_Keyword_25", "Medical_Keyword_26", "Medical_Keyword_27", "Medical_Keyword_28", "Medical_Keyword_29", "Medical_Keyword_30", "Medical_Keyword_31", "Medical_Keyword_32", "Medical_Keyword_33", "Medical_Keyword_34", "Medical_Keyword_35", "Medical_Keyword_36", "Medical_Keyword_37", "Medical_Keyword_38", "Medical_Keyword_39", "Medical_Keyword_40", "Medical_Keyword_41", "Medical_Keyword_42", "Medical_Keyword_43", "Medical_Keyword_44", "Medical_Keyword_45", "Medical_Keyword_46", "Medical_Keyword_47", "Medical_Keyword_48", "Product_Info_2_stringindexer"], "Response")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

